In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Hybrid Model BELOW:

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Dense, Conv1D, LSTM, Dropout,
                                     TimeDistributed, Flatten)

# Load and preprocess data
def load_file(filepath):
    return np.loadtxt(filepath)

def load_data_train(data_folder):
    filepaths = [
        f"{data_folder}/body_acc_x_train.txt", f"{data_folder}/body_acc_y_train.txt", f"{data_folder}/body_acc_z_train.txt",
        f"{data_folder}/body_gyro_x_train.txt", f"{data_folder}/body_gyro_y_train.txt", f"{data_folder}/body_gyro_z_train.txt",
        f"{data_folder}/total_acc_x_train.txt", f"{data_folder}/total_acc_y_train.txt", f"{data_folder}/total_acc_z_train.txt"
    ]
    signals = [load_file(path) for path in filepaths]
    X = np.stack(signals, axis=-1)  # Shape: (samples, timesteps, features)
    return X

def load_data_test(data_folder):
    filepaths = [
        f"{data_folder}/body_acc_x_test.txt", f"{data_folder}/body_acc_y_test.txt", f"{data_folder}/body_acc_z_test.txt",
        f"{data_folder}/body_gyro_x_test.txt", f"{data_folder}/body_gyro_y_test.txt", f"{data_folder}/body_gyro_z_test.txt",
        f"{data_folder}/total_acc_x_test.txt", f"{data_folder}/total_acc_y_test.txt", f"{data_folder}/total_acc_z_test.txt"
    ]
    signals = [load_file(path) for path in filepaths]
    X = np.stack(signals, axis=-1)  # Shape: (samples, timesteps, features)
    return X

def preprocess_labels(label_filepath):
    y = np.loadtxt(label_filepath).astype(int) - 1  # Convert to 0-based indexing
    y = tf.keras.utils.to_categorical(y)  # One-hot encoding
    return y

# Load training and test data
train_data_folder = "/content/drive/MyDrive/Inertial_Signals_train"
test_data_folder = "/content/drive/MyDrive/Inertial_Signals_test"
X_train = load_data_train(train_data_folder)  # (7352, 128, 9)
X_test = load_data_test(test_data_folder)    # (2947, 128, 9)
y_train = preprocess_labels("/content/drive/MyDrive/Inertial_Signals_train/y_train.txt")  # (7352, 6)
y_test = preprocess_labels("/content/drive/MyDrive/Inertial_Signals_test/y_test.txt")     # (2947, 6)


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam

# Load data (assuming X_train, X_test, y_train, y_test are already preprocessed)
# Example data shape: (7352, 128, 9) for X_train and (7352, 6) for y_train

# Define the Hybrid CNN-LSTM Model
model = Sequential()

# Step 1: Add Conv1D Layer (CNN part)
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(MaxPooling1D(pool_size=2))

# Step 2: Add another Conv1D Layer (to capture higher-level features)
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# Step 3: Add an LSTM Layer (LSTM part)
model.add(LSTM(100, return_sequences=False))  # Set return_sequences=True if you want to stack LSTMs

# Step 4: Add Dropout for Regularization
model.add(Dropout(0.5))

# Step 5: Flatten the output (if needed) and connect to Dense layers
model.add(Flatten())

# Step 6: Add a Dense Layer for Classification
model.add(Dense(64, activation='relu'))

# Step 7: Output Layer with Softmax Activation for Multi-Class Classification
model.add(Dense(6, activation='softmax'))  # Assuming 6 classes in the output

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary to inspect architecture
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_23 (Conv1D)                   │ (None, 126, 64)             │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 63, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_24 (Conv1D)                   │ (None, 61, 128)             │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 30, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_32 (LSTM)                       │ (None, 100)                 │          91,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_25 (Dropout)                 │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_16 (Flatten)                 │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 64)                  │           6,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 6)                   │             390 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 124,950 (488.09 KB)

 Trainable params: 124,950 (488.09 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Assuming you have preprocessed X_train, X_test, y_train, y_test
# X_train shape should be (7352, 128, 9), y_train shape should be (7352, 6)

history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))


Epoch 1/20
115/115 ━━━━━━━━━━━━━━━━━━━━ 14s 93ms/step - accuracy: 0.5563 - loss: 1.0649 - val_accuracy: 0.7391 - val_loss: 0.6086
Epoch 2/20
115/115 ━━━━━━━━━━━━━━━━━━━━ 21s 100ms/step - accuracy: 0.8357 - loss: 0.4024 - val_accuracy: 0.8527 - val_loss: 0.4102
Epoch 3/20
115/115 ━━━━━━━━━━━━━━━━━━━━ 18s 83ms/step - accuracy: 0.9250 - loss: 0.2047 - val_accuracy: 0.8809 - val_loss: 0.4106
Epoch 4/20
115/115 ━━━━━━━━━━━━━━━━━━━━ 11s 98ms/step - accuracy: 0.9453 - loss: 0.1462 - val_accuracy: 0.8870 - val_loss: 0.3841
Epoch 5/20
115/115 ━━━━━━━━━━━━━━━━━━━━ 21s 104ms/step - accuracy: 0.9445 - loss: 0.1251 - val_accuracy: 0.8890 - val_loss: 0.4017
Epoch 6/20
115/115 ━━━━━━━━━━━━━━━━━━━━ 9s 80ms/step - accuracy: 0.9497 - loss: 0.1164 - val_accuracy: 0.8775 - val_loss: 0.4057
Epoch 7/20
115/115 ━━━━━━━━━━━━━━━━━━━━ 11s 98ms/step - accuracy: 0.9478 - loss: 0.1192 - val_accuracy: 0.8599 - val_loss: 0.4876
Epoch 8/20
115/115 ━━━━━━━━━━━━━━━━━━━━ 21s 100ms/step - accuracy: 0.9516 - loss: 0.1144 

In [ ]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.8727 - loss: 0.4715
Test Loss: 0.4111708104610443
Test Accuracy: 0.9117746949195862


In [ ]:
import numpy as np
from sklearn.metrics import f1_score, confusion_matrix

# After training the model, make predictions on the test set
predictions = model.predict(X_test)  # Shape: (num_samples, num_classes)

# Get the predicted labels by choosing the class with the highest probability (argmax)
predicted_labels = np.argmax(predictions, axis=1)

# Get the true labels (y_test is already one-hot encoded)
true_labels = np.argmax(y_test, axis=1)  # Convert one-hot encoded labels to class labels

# Compute the F1 score
f1 = f1_score(true_labels, predicted_labels, average='weighted')
print(f"F1 Score: {f1:.2f}")

# Compute and print the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:")
print(cm)


NameError: name 'model' is not defined

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Assume true_labels and predicted_labels are already defined
# true_labels = np.argmax(y_test, axis=1)
# predicted_labels = np.argmax(predictions, axis=1)

# Compute the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Create the heatmap plot
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False, xticklabels=np.arange(1, 7), yticklabels=np.arange(1, 7))

# Add labels and title
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix for CNN-LSTM')

# Show the plot
plt.show()


NameError: name 'true_labels' is not defined